In [1]:
import pandas as pd
import numpy as np


In [2]:
#Read trainig and testing files
train = pd.read_csv("train_kOBLwZA (1).csv")
test = pd.read_csv("test_t02dQwI.csv")
data = pd.concat([train, test], sort = False)

In [3]:
train.shape,test.shape, data.shape

((8523, 12), (5681, 11), (14204, 12))

In [4]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [5]:
#Finding the missing values
data.isnull().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
dtype: int64

### Imputation

#### Outlet_Size

In [6]:
data.Outlet_Size.fillna('None', inplace = True)

#### Item_Weight

In [7]:
data.Item_Weight.fillna(data.Item_Weight.mean(), inplace = True)

#### One Hot Encoding

In [8]:
column = ['Item_Fat_Content','Item_Type','Outlet_Size',
          'Outlet_Location_Type','Outlet_Type']
OneHotEncoding1 = pd.get_dummies(data[column])
OneHotEncoding2 = pd.get_dummies(data.Outlet_Establishment_Year)
OneHotEncoding = pd.concat([OneHotEncoding2,OneHotEncoding1], axis =1, sort = False)
OneHotEncoding.head(3)

,1985,1987,1997,1998,1999,2002,2004,2007,2009,Item_Fat_Content_LF,...,Outlet_Size_Medium,Outlet_Size_None,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,1,0
2,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,0


In [9]:
data = data[['Item_Identifier','Outlet_Identifier','Item_MRP','Item_Visibility','Item_Outlet_Sales']]

In [10]:
total_df = pd.concat([data, OneHotEncoding],axis = 1, sort =False)

In [11]:
total_df.head(3)

,Item_Identifier,Outlet_Identifier,Item_MRP,Item_Visibility,Item_Outlet_Sales,1985,1987,1997,1998,1999,...,Outlet_Size_Medium,Outlet_Size_None,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,FDA15,OUT049,249.8092,0.016047,3735.1380,0,0,0,0,1,...,1,0,0,1,0,0,0,1,0,0
1,DRC01,OUT018,48.2692,0.019278,443.4228,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
2,FDN15,OUT049,141.6180,0.016760,2097.2700,0,0,0,0,1,...,1,0,0,1,0,0,0,1,0,0


In [12]:
train = total_df[:len(train)]
test = total_df[len(train):]
X_train=train.drop(['Item_Outlet_Sales','Item_Identifier','Outlet_Identifier'],axis=1)
y_train=train['Item_Outlet_Sales']
d = test[['Item_Identifier','Outlet_Identifier']]
test = test.drop(['Item_Outlet_Sales','Item_Identifier','Outlet_Identifier'],axis=1)
test.shape, X_train.shape

((5681, 43), (8523, 43))

In [13]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

# rc=RobustScaler()
# X_train=rc.fit_transform(X_train)
# df_Test=rc.transform(test)

sc=StandardScaler()
X_train=sc.fit_transform(X_train)
df_Test=sc.transform(test)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=0.7, random_state=1234)

In [15]:
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from math import sqrt

# fit model no training data
model = XGBRegressor()

In [2]:
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

## Hyper Parameter Optimization
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [4]:
# Set up the random search with 4-fold cross validation
from sklearn.model_selection import RandomizedSearchCV
random_cv = RandomizedSearchCV(estimator=model,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [18]:
random_cv.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   29.5s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  4.7min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  9.5min
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed: 15.1min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_con...
                   iid='warn', n_iter=50, n_jobs=4,
                   param_distributions={'base_score': [0.25, 0.5, 0.75, 1],
                                        'booster': ['gbtree', 'gblinear'],
                              

In [19]:
random_cv.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [20]:
regressor=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [21]:
regressor.fit(X_train,y_train)

[17:05:22] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [22]:

# make predictions for test data
y_pred = regressor.predict(X_validation)

# evaluate predictions
rmse = sqrt(mean_squared_error(y_validation, y_pred))
print("Accuracy on test:", rmse)

# make predictions for train data
y_pred = regressor.predict(X_train)

# evaluate predictions
rmse = sqrt(mean_squared_error(y_train, y_pred))
print("Accuracy on train:", (rmse))

Accuracy on test: 1099.4746395033744
Accuracy on train: 1048.2826738330857


In [23]:
# Accuracy on test: 1062.4118370693648
# Accuracy on train: 1046.8766457627478

In [24]:
submission = pd.DataFrame()
submission['Item_Identifier'] = d['Item_Identifier']
submission['Outlet_Identifier'] = d['Outlet_Identifier']
submission['Item_Outlet_Sales'] = model.predict(df_Test)
submission.to_csv("Submission.csv", index=False)

XGBoostError: need to call fit or load_model beforehand

In [ ]:
df = pd.read_fwf('SampleSubmission_TmnO39y.txt',delimiter="\t")